In [1]:
#imports
from scapy.all import *
from scapy.layers.http import HTTPRequest
from scapy.layers.dns import DNS, DNSRR
from scapy.layers.inet import IP, TCP
from scapy.layers.dhcp import DHCP
from scapy.layers.netbios import NBNSQueryRequest
import ipaddress

In [2]:
def is_private_ip(ip_str):
    try:
        ip = ipaddress.ip_address(ip_str)
        return ip.is_private
    except ValueError:
        return False

In [3]:
packets = rdpcap('6.pcap')

In [4]:
# 1. Find Attacker's Phishing Page IP
phishing_ip = None
# Check DNS responses for secure-bank.com pointing to a private IP
for pkt in packets:
    if pkt.haslayer(DNSRR):
        dns = pkt[DNS]
        if dns.qr == 1:  # DNS Response
            qname = dns.qd.qname.decode().lower() if dns.qd else ''
            if 'secure-bank.com' in qname:
                for rr in dns.an:
                    if rr.type == 1:  # A record
                        answer_ip = rr.rdata if not isinstance(rr.rdata, bytes) else rr.rdata.decode()
                        if is_private_ip(answer_ip):
                            phishing_ip = answer_ip
                            break
                if phishing_ip:
                    break
phishing_ip

'192.168.1.100'

In [28]:
# 2. Extract Victim's Username from Plaintext POST Request
from urllib.parse import parse_qs, unquote
username = None
if phishing_ip:
    for pkt in packets:
        # Check for HTTP traffic to phishing IP
        if pkt.haslayer(IP) and pkt[IP].dst == phishing_ip:
            # Handle both HTTP Request and Response layers
            if pkt.haslayer(HTTPRequest):
                http = pkt[HTTPRequest]
                # Detect POST requests more reliably
                if b'POST' in http.Method:
                    # Get full payload from Raw layer
                    if pkt.haslayer(Raw):
                        load = pkt[Raw].load.decode('utf-8', errors='ignore')
                        # Find POST parameters in body
                        body = load.split('\r\n\r\n')[-1]
                        # Try URL-encoded format
                        if 'username=' in body:
                            params = parse_qs(body)
                            if 'username' in params:
                                username = unquote(params['username'][0])
                                password = unquote(params['password'][0])
                                break
                        
                        # Try JSON format
                        if not username and ('{' in body and '}' in body):
                            match = re.search(r'"username"\s*:\s*"([^"]+)"', body)
                            if match:
                                username = unquote(match.group(1))
                                break

username, password

('alexwell', 'bemymaxwell')

In [12]:
def analyze_attacker_packets():
    
    attacker_info = {
        'email_address': None,
        'email_subject': None,
        'email_body': None,
        'hostname': None,
    }

    for pkt in packets:
        if pkt.haslayer(IP) and pkt[IP].src == phishing_ip:
            # Check SMTP traffic
            if pkt.haslayer(TCP) and pkt[TCP].dport == 25 and pkt.haslayer(Raw):
                payload = pkt[Raw].load.decode()
                if 'MAIL FROM:' in payload:
                    attacker_info['email'] = payload.split('MAIL FROM:<')[1].split('>')[0]
                if 'HELO' in payload:
                    attacker_info['hostname'] = payload.split('HELO ')[1].split('\r\n')[0]
                if 'Subject' in payload:
                    attacker_info['email_subject'] = payload.split('Subject: ')[1].split('\r\n')[0]
                attacker_info['email_body'] = payload.split('\r\n\r\n')[1].split('\r\n.\r\n')[0]

    return attacker_info

In [29]:
result = analyze_attacker_packets()

In [33]:
print("Answers to the questions:")
print("1. IP address of the phishing page: ", phishing_ip)
print("2. username and password: of the victim are", username, password)
print("3. Information about the attacker:")
print("\tName of the attacker: ", result['hostname'])
print("\tEmail address of the attacker: ", result['email'])
print("\tEmail subject: ", result['email_subject'])
print("\tEmail body: ", result['email_body'])

Answers to the questions:
1. IP address of the phishing page:  192.168.1.100
2. username and password: of the victim are alexwell bemymaxwell
3. Information about the attacker:
	Name of the attacker:  Chris Martin
	Email address of the attacker:  ab@iitgn.ac.in
	Email subject:  CS331 - Enrollment
	Email body:  Can I get enrolled into CS331? I need it for my graduation.
